### Exercise 1

- dataset: https://archive.ics.uci.edu/dataset/53/iris
- task: use the Iris dataset (binary classification case, so you can limit it to only two classes, e.g., Setosa vs. Versicolor) and perform logistic regression with 30% test data to classify the flowers into two categories based on their features (sepal and petal lengths and widths).

### Exercise 2

- dataset: https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic
- task: use the Breast Cancer dataset to predict cancer diagnosis.

### Exercise 3

- dataset: returns.csv
- task: use the features Stock_1 and Stock_2 to classify whether Stock_3 went up on that day.

### Exercise 4

- dataset: simpson_paradox.csv
- task 1: use linear regression to find a model for the col1 and col2 features
- task 2: try combining linear regression and logistic regression to improve your model

In [4]:
# For exercises 5, 6, and 7, your pipeline might look something like:
"""
model = Pipeline([
    ("scaler", StandardScaler()),
    ("rbf_transform", rbf_feature),  # your transform here
    ("log_reg", LogisticRegression())
])
"""

'\nmodel = Pipeline([\n    ("scaler", StandardScaler()),\n    ("rbf_transform", rbf_feature),  # your transform here\n    ("log_reg", LogisticRegression())\n])\n'

### Exercise 5

- dataset: ring_data.csv
- task 1: find a map phi: R^2 -> R^3 that separates your datapoints
- task 2: train a logistic regression (linear model) on your transformed data
- task 3: Use scikit-learn PolynomialFeatures(degree=2) and train a logistic regression on the transformed data
- task 4: Use scikit-learn RBFSampler(gamma=1.0, n_components=100, random_state=42) and train a logistic regression on the transformed data

Measure your classifications using the accuracy metric

### Exercise 6

- dataset: moon_data.csv
- task: Use scikit-learn RBFSampler(gamma=1.0, n_components=100, random_state=42) and train a logistic regression on the transformed data

Measure your classifications using the accuracy metric

### Exercise 7

- dataset: spiral_data.csv
- task: Use scikit-learn RBFSampler(gamma=1.0, n_components=100, random_state=42) and train a logistic regression on the transformed data

Measure your classifications using the accuracy metric

### Exercise 8

- task: implement a KernelLinearRegression from scratch. The user should be able to select a kernel function to be used inside the sigmoid activation function. Test it on one of the above dataset and compare with the coresponding exercise.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from scipy.special import expit

class KernelLogisticRegression:
    def __init__(self, kernel="rbf", gamma=1.0, degree=3, lambda_reg=1e-3, lr=0.1, max_iter=500):
        """
        Kernelized Logistic Regression using Gradient Descent.
        :param kernel: "linear", "polynomial", or "rbf"
        :param gamma: RBF kernel parameter
        :param degree: Polynomial kernel degree
        :param lambda_reg: Regularization parameter
        :param lr: Learning rate for gradient descent
        :param max_iter: Maximum number of iterations
        """
        self.kernel = kernel
        self.gamma = gamma
        self.degree = degree
        self.lambda_reg = lambda_reg
        self.lr = lr
        self.max_iter = max_iter
        self.alpha = None
        self.X_train = None

    def _compute_kernel(self, X1, X2):
        """ Compute the Kernel Matrix K(X1, X2) """
        if self.kernel == "linear":
            return X1 @ X2.T  # K(x, y) = ⟨x, y⟩
        
        elif self.kernel == "polynomial":
            return (1 + X1 @ X2.T) ** self.degree  # K(x, y) = (1 + ⟨x, y⟩)^d
        
        elif self.kernel == "rbf":
            sq_dists = np.sum(X1**2, axis=1, keepdims=True) - 2 * X1 @ X2.T + np.sum(X2**2, axis=1)
            return np.exp(-self.gamma * sq_dists)  # K(x, y) = exp(-γ||x - y||^2)
        
        else:
            raise ValueError("Unknown kernel type. Choose 'linear', 'polynomial', or 'rbf'.")

    def sigmoid(self, z):
        """ Sigmoid function: σ(z) = 1 / (1 + exp(-z)) """
        return expit(z)

    def fit(self, X, y):
        """ Train Kernel Logistic Regression using Gradient Descent """
        self.X_train = X
        K = self._compute_kernel(X, X)
        n = K.shape[0]
        
        self.alpha = np.zeros(n)
        for _ in range(self.max_iter):
            y_pred = self.sigmoid(K @ self.alpha)
            gradient = K.T @ (y_pred - y) + self.lambda_reg * self.alpha
            self.alpha -= self.lr * gradient

    def predict_proba(self, X):
        """ Predict probability estimates using sigmoid activation """
        K_test = self._compute_kernel(X, self.X_train)
        return self.sigmoid(K_test @ self.alpha)

    def predict(self, X):
        """ Predict class labels (0 or 1) """
        return (self.predict_proba(X) >= 0.5).astype(int)